In [1]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Sports_and_Outdoors.csv

--2021-03-02 11:10:07--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/ratings_Sports_and_Outdoors.csv
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133182998 (127M) [text/csv]
Saving to: ‘ratings_Sports_and_Outdoors.csv’

ratings_Sports_and_ 100%[===================>] 127.01M  1002KB/s    in 1m 48s  

2021-03-02 11:11:54 (1.18 MB/s) - ‘ratings_Sports_and_Outdoors.csv’ saved [133182998/133182998]



In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Sports_and_Outdoors.json.gz

--2021-03-02 11:11:55--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles/meta_Sports_and_Outdoors.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 512767929 (489M) [application/octet-stream]
Saving to: ‘meta_Sports_and_Outdoors.json.gz’

meta_Sports_and_Out 100%[===================>] 489.01M  42.3MB/s    in 12s     

2021-03-02 11:12:07 (40.7 MB/s) - ‘meta_Sports_and_Outdoors.json.gz’ saved [512767929/512767929]



In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split
import gzip
#import Recommender as Recommender
#import Evaluation as Evaluation

In [4]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

dfmeta = getDF('./meta_Sports_and_Outdoors.json.gz')

In [5]:
dfmeta.head()

,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,details
0,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Adult Tutu Assorted Colors (Turquoise),[],[],,BubuBibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","712,899 in Clothing, Shoes & Jewelry (","[B071LQWQBQ, B00M14DG0O, B076GTTY9W, B07BSZ4GY...","<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.80,0000032042,NaN
1,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Bububibi Adult Ballet Tutu Cheetah Pink,[],[],,BubuBibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","712,899 in Clothing, Shoes & Jewelry (",[],"<img src=""https://images-na.ssl-images-amazon....",,"<div class=""a-fixed-left-grid a-spacing-none"">...",$11.97,0000032069,NaN
2,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[3 layers of super-soft polyester tulle can be...,,Girls Ballet Tutu Neon Orange,[B01MYHD3LV],[],,Unknown,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","[>#924,198 in Toys & Games (See Top 100 in Toy...","[B0152HBC52, B07C29WWV1, B01M0BIP9H, B00VXJHFR...",Toys & Games,,,$5.70,0000031860,NaN
3,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[TUtu],,Girls Ballet Tutu Zebra Hot Pink,[],[],,Tutu,[Tutu],"1,314,199 in Sports & Outdoors (",[0000013714],Sports & Outdoors,,,$7.50,0000031852,NaN
4,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[Dance tutu for girls ages 2-8 years. Perfect ...,,Girls Ballet Tutu Neon Blue,"[B009THXHPA, B00BEU1ZBI, B079PYLZTM, B00BEU1ZR...",[],,Bububibi,"[3 Layers - 100% Polyester Tulle, Hand Wash La...","[>#393,597 in Toys & Games (See Top 100 in Toy...","[B009THXHPA, B079PYLZTM, B079PQPLT3, B00UP6A3R...",Toys & Games,,,$6.54,0000031895,NaN


In [6]:
dfmeta.shape

(962300, 18)

In [7]:
dfreviews=pd.read_csv('./ratings_Sports_and_Outdoors.csv', header=None,sep=',')

In [8]:
dfreviews.head()

,0,1,2,3
0,A3PMSRCL80KSA1,0000031852,4.0,1388275200
1,A1SNLWGLFXD70K,0000031852,4.0,1392940800
2,A1KJ4CVG87QW09,0000031852,4.0,1389657600
3,AA9ITO6ZLZW6,0000031852,5.0,1399507200
4,APJ5ULJ1RMZ4,0000031852,1.0,1398556800


In [9]:
dfreviews.columns = ["reviewerID", "asin","overall","timestamp"]
dfreviews.head()

,reviewerID,asin,overall,timestamp
0,A3PMSRCL80KSA1,0000031852,4.0,1388275200
1,A1SNLWGLFXD70K,0000031852,4.0,1392940800
2,A1KJ4CVG87QW09,0000031852,4.0,1389657600
3,AA9ITO6ZLZW6,0000031852,5.0,1399507200
4,APJ5ULJ1RMZ4,0000031852,1.0,1398556800


In [10]:
dfreviews.shape

(3268695, 4)

In [11]:
product_ratings=pd.merge(dfreviews,dfmeta,on='asin',how='inner')

In [12]:
product_ratings.head()

,reviewerID,asin,overall,timestamp,category,tech1,description,fit,title,also_buy,image,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,details
0,A3PMSRCL80KSA1,0000031852,4.0,1388275200,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[TUtu],,Girls Ballet Tutu Zebra Hot Pink,[],[],,Tutu,[Tutu],"1,314,199 in Sports & Outdoors (",[0000013714],Sports & Outdoors,,,$7.50,NaN
1,A1SNLWGLFXD70K,0000031852,4.0,1392940800,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[TUtu],,Girls Ballet Tutu Zebra Hot Pink,[],[],,Tutu,[Tutu],"1,314,199 in Sports & Outdoors (",[0000013714],Sports & Outdoors,,,$7.50,NaN
2,A1KJ4CVG87QW09,0000031852,4.0,1389657600,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[TUtu],,Girls Ballet Tutu Zebra Hot Pink,[],[],,Tutu,[Tutu],"1,314,199 in Sports & Outdoors (",[0000013714],Sports & Outdoors,,,$7.50,NaN
3,AA9ITO6ZLZW6,0000031852,5.0,1399507200,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[TUtu],,Girls Ballet Tutu Zebra Hot Pink,[],[],,Tutu,[Tutu],"1,314,199 in Sports & Outdoors (",[0000013714],Sports & Outdoors,,,$7.50,NaN
4,APJ5ULJ1RMZ4,0000031852,1.0,1398556800,"[Sports & Outdoors, Sports & Fitness, Other Sp...",,[TUtu],,Girls Ballet Tutu Zebra Hot Pink,[],[],,Tutu,[Tutu],"1,314,199 in Sports & Outdoors (",[0000013714],Sports & Outdoors,,,$7.50,NaN


In [13]:
product_ratings.shape

(1533315, 21)

In [14]:
product_ratings.drop(['timestamp','description','category','tech1','fit','also_buy','tech2','brand','feature','rank','also_view','details','main_cat','similar_item','date','price'], axis=1, inplace=True)

In [15]:
#product_ratings.drop(['image'],axis=1,inplace=True)

In [16]:
product_ratings.head()

,reviewerID,asin,overall,title,image
0,A3PMSRCL80KSA1,0000031852,4.0,Girls Ballet Tutu Zebra Hot Pink,[]
1,A1SNLWGLFXD70K,0000031852,4.0,Girls Ballet Tutu Zebra Hot Pink,[]
2,A1KJ4CVG87QW09,0000031852,4.0,Girls Ballet Tutu Zebra Hot Pink,[]
3,AA9ITO6ZLZW6,0000031852,5.0,Girls Ballet Tutu Zebra Hot Pink,[]
4,APJ5ULJ1RMZ4,0000031852,1.0,Girls Ballet Tutu Zebra Hot Pink,[]


In [17]:
product_ratings.shape

(1533315, 5)

In [18]:
product_ratings = product_ratings.rename(columns={'reviewerID': 'userID', 'asin': 'prod_ID','title': 'prod_name',
                                                 'overall': 'rating' ,'image' : 'imgurl'})

In [19]:
product_ratings.head()

,userID,prod_ID,rating,prod_name,imgurl
0,A3PMSRCL80KSA1,0000031852,4.0,Girls Ballet Tutu Zebra Hot Pink,[]
1,A1SNLWGLFXD70K,0000031852,4.0,Girls Ballet Tutu Zebra Hot Pink,[]
2,A1KJ4CVG87QW09,0000031852,4.0,Girls Ballet Tutu Zebra Hot Pink,[]
3,AA9ITO6ZLZW6,0000031852,5.0,Girls Ballet Tutu Zebra Hot Pink,[]
4,APJ5ULJ1RMZ4,0000031852,1.0,Girls Ballet Tutu Zebra Hot Pink,[]


In [20]:
product_ratings.isnull().sum()

userID       0
prod_ID      0
rating       0
prod_name    0
imgurl       0
dtype: int64

In [21]:
df=product_ratings

In [22]:
df.dropna(inplace=True)

In [23]:
counts1=df['userID'].value_counts() 
counts=df['prod_ID'].value_counts()

In [24]:
df1=df[df['userID'].isin(counts1[counts1 >=50].index)]
df1.shape

(3158, 5)

In [25]:
df1=df1[df1['prod_ID'].isin(counts[counts >=50].index)]
df1.shape

(1208, 5)

In [26]:
df1.drop_duplicates(subset=['userID','prod_ID'],inplace=True)

In [27]:
df1.shape

(1162, 5)

In [28]:
df1.head()

,userID,prod_ID,rating,prod_name,imgurl
3821,A1M2T0J45TTE64,B00004S9I0,5.0,Lodge Original Finish Pro Grid/Iron Griddle,[https://images-na.ssl-images-amazon.com/image...
4937,AGGAQLXPVGM5D,B00004U9J9,5.0,SOG Pentagon Elite I Folding Knife PE14-CP - B...,[https://images-na.ssl-images-amazon.com/image...
6679,AK7CAW3N1XZV6,B0000534W2,5.0,"TheraBand Exercise Ball, Stability Ball with 6...",[https://images-na.ssl-images-amazon.com/image...
7753,AGGAQLXPVGM5D,B00005AXIY,5.0,Bushnell Yardage Pro Scout Laser Rangefinder,[https://images-na.ssl-images-amazon.com/image...
9663,A22CW0ZHY3NJH8,B00005BV0D,5.0,Liberty Mountain Aerobie Sprint,[https://images-na.ssl-images-amazon.com/image...


In [29]:
df1['prod_ID'].nunique()

874

In [30]:
df1['userID'].nunique()

45

In [31]:
ratingsd=df1.pivot(index='userID',columns= 'prod_ID',values='rating').fillna(0)
ratingsd.head()

prod_ID,B00004S9I0,B00004U9J9,B0000534W2,B00005AXIY,B00005BV0D,B00005BXUU,B000068CJZ,B0000789T2,B00007E1M2,B00008LDZZ,B00009363P,B0000A0AHF,B0000B0ETV,B0000C53IH,B0000CA2PA,B0000V0L9Y,B0000V2DNG,B0000WR6W8,B00012345A,B00012M5MS,B0001EJEKM,B0001NDYOK,B0001P151W,B0001W0E7G,B0001WC898,B00022KJ16,B00024JMWG,B0002558CS,B000265SMW,B00026P3QS,B00029EWYY,B0002CQR08,B0002E86WS,B0002IKE8I,B0002MBPDC,B000650YAU,B0006519ZE,B00065X222,B00066U2YM,B00066ZXO6,...,B009GDKXZ4,B009WGFYRC,B009YSFT7S,B00AB63708,B00ADUYW9M,B00AHU76AK,B00AOZHW3O,B00AP311JQ,B00APP5AUU,B00AU6NJLG,B00B1MZ22C,B00B1N0R6C,B00B4FY8YO,B00BCKOKD0,B00BMTMPFQ,B00BPHNIZG,B00BQ0QBME,B00BQT69PO,B00BS2O39I,B00BS4RP7S,B00BV6OXRI,B00C81JUS2,B00C81JUSM,B00DII7CTI,B00DQ1RDWS,B00DVE8DUG,B00DVM34NO,B00EAHYZD2,B00FA2RLX2,B00GLL98Y6,B00GV0TGQW,B00HA2ZKCO,B00I0HQ4MS,B00I8XZ4HA,B00ILI7SVW,B00IOGK4RG,B00J2QLCFU,B00J39OM2Q,B00J5HDEPC,B00JQDFSGS
userID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A16Z9OKB0NVI3F,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A18ECVX2RJ7HUE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A19XC6Z95GK5AB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
A19YS2UWGDR6WM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1M2T0J45TTE64,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
ratingsd.shape

(45, 874)

In [33]:
corrMatrix = ratingsd.corr(method='pearson')
corrMatrix.head(100)

prod_ID,B00004S9I0,B00004U9J9,B0000534W2,B00005AXIY,B00005BV0D,B00005BXUU,B000068CJZ,B0000789T2,B00007E1M2,B00008LDZZ,B00009363P,B0000A0AHF,B0000B0ETV,B0000C53IH,B0000CA2PA,B0000V0L9Y,B0000V2DNG,B0000WR6W8,B00012345A,B00012M5MS,B0001EJEKM,B0001NDYOK,B0001P151W,B0001W0E7G,B0001WC898,B00022KJ16,B00024JMWG,B0002558CS,B000265SMW,B00026P3QS,B00029EWYY,B0002CQR08,B0002E86WS,B0002IKE8I,B0002MBPDC,B000650YAU,B0006519ZE,B00065X222,B00066U2YM,B00066ZXO6,...,B009GDKXZ4,B009WGFYRC,B009YSFT7S,B00AB63708,B00ADUYW9M,B00AHU76AK,B00AOZHW3O,B00AP311JQ,B00APP5AUU,B00AU6NJLG,B00B1MZ22C,B00B1N0R6C,B00B4FY8YO,B00BCKOKD0,B00BMTMPFQ,B00BPHNIZG,B00BQ0QBME,B00BQT69PO,B00BS2O39I,B00BS4RP7S,B00BV6OXRI,B00C81JUS2,B00C81JUSM,B00DII7CTI,B00DQ1RDWS,B00DVE8DUG,B00DVM34NO,B00EAHYZD2,B00FA2RLX2,B00GLL98Y6,B00GV0TGQW,B00HA2ZKCO,B00I0HQ4MS,B00I8XZ4HA,B00ILI7SVW,B00IOGK4RG,B00J2QLCFU,B00J39OM2Q,B00J5HDEPC,B00JQDFSGS
prod_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
B00004S9I0,1.000000,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.032305,-0.022727,-0.022727,-0.032171,-0.022727,1.000000,-0.038668,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040291,-0.022727,0.564076,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.046772,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.022727,-0.032305,-0.022727,-0.032513,-0.022727,-0.047088,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040073,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,-0.032171,-0.040291,-0.022727,-0.022727,-0.022727
B00004U9J9,-0.022727,1.000000,-0.022727,1.000000,-0.022727,-0.022727,-0.022727,-0.032305,-0.022727,-0.022727,-0.032171,-0.022727,-0.022727,-0.038668,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040291,-0.022727,-0.040291,-0.022727,-0.022727,1.000000,-0.022727,-0.022727,-0.022727,-0.022727,-0.046772,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.022727,-0.032305,-0.022727,-0.032513,-0.022727,-0.047088,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040073,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,-0.032171,-0.040291,-0.022727,-0.022727,-0.022727
B0000534W2,-0.022727,-0.022727,1.000000,-0.022727,-0.022727,-0.022727,-0.022727,-0.032305,-0.022727,1.000000,-0.032171,-0.022727,-0.022727,-0.038668,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040291,-0.022727,-0.040291,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.046772,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.022727,-0.032305,-0.022727,-0.032513,-0.022727,-0.047088,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,0.603953,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,0.588271,0.564076,-0.022727,-0.022727,-0.022727
B00005AXIY,-0.022727,1.000000,-0.022727,1.000000,-0.022727,-0.022727,-0.022727,-0.032305,-0.022727,-0.022727,-0.032171,-0.022727,-0.022727,-0.038668,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040291,-0.022727,-0.040291,-0.022727,-0.022727,1.000000,-0.022727,-0.022727,-0.022727,-0.022727,-0.046772,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.022727,-0.032305,-0.022727,-0.032513,-0.022727,-0.047088,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.

In [34]:
corrMatrix.shape

(874, 874)

In [35]:
def get_similar(prod_id,rating):
    similar_ratings = corrMatrix[prod_id]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [36]:
def prod_name(prodid):
    return df1.loc[df1['prod_ID'] == prodid ].iloc[0]['prod_name']

In [37]:
def prod_img(prodid):
    return df1.loc[df1['prod_ID'] == prodid ].iloc[0]['imgurl']

In [38]:
items = [("7245456313",3),("B000002NUS",1),("B00019NKES",2)]
similar_products = pd.DataFrame()
for product,rating in items:
    similar_products = similar_products.append(get_similar(product,rating),ignore_index = True)

similar_products.head(10)

KeyError: ignored

In [39]:
final_results = pd.DataFrame(similar_products.sum().sort_values(ascending=False).head(20))
final_results.reset_index(inplace = True)
final_results.columns = ['prodid','rating']

In [ ]:
def recommendations(items):
    similar_products = pd.DataFrame()
    for product,rating in items:
        similar_products = similar_products.append(get_similar(product,rating),ignore_index = True)

    final_results = pd.DataFrame(similar_products.sum().sort_values(ascending=False).head(20))
    final_results.reset_index(inplace = True)
    final_results.columns = ['prodid','rating']
    final_results['prodname'] = final_results['prodid'].apply(prod_name)
    final_results['imgurl'] = final_results['prodid'].apply(prod_img)
    
    return final_results

In [40]:
def original_product(items):
    original_products = pd.DataFrame()
    for product,rating in items:
           print(prod_name(product))

In [ ]:
items = [("7245456313",3),("B000002NUS",1),("B00019NKES",2)]
original_product(items)
recommendations(items)

In [41]:
corrMatrix.to_csv('./prod_similarity_sports.csv')

In [42]:
corrdf = pd.read_csv('./prod_similarity_sports.csv')

In [43]:
corrdf.head()

,prod_ID,B00004S9I0,B00004U9J9,B0000534W2,B00005AXIY,B00005BV0D,B00005BXUU,B000068CJZ,B0000789T2,B00007E1M2,B00008LDZZ,B00009363P,B0000A0AHF,B0000B0ETV,B0000C53IH,B0000CA2PA,B0000V0L9Y,B0000V2DNG,B0000WR6W8,B00012345A,B00012M5MS,B0001EJEKM,B0001NDYOK,B0001P151W,B0001W0E7G,B0001WC898,B00022KJ16,B00024JMWG,B0002558CS,B000265SMW,B00026P3QS,B00029EWYY,B0002CQR08,B0002E86WS,B0002IKE8I,B0002MBPDC,B000650YAU,B0006519ZE,B00065X222,B00066U2YM,...,B009GDKXZ4,B009WGFYRC,B009YSFT7S,B00AB63708,B00ADUYW9M,B00AHU76AK,B00AOZHW3O,B00AP311JQ,B00APP5AUU,B00AU6NJLG,B00B1MZ22C,B00B1N0R6C,B00B4FY8YO,B00BCKOKD0,B00BMTMPFQ,B00BPHNIZG,B00BQ0QBME,B00BQT69PO,B00BS2O39I,B00BS4RP7S,B00BV6OXRI,B00C81JUS2,B00C81JUSM,B00DII7CTI,B00DQ1RDWS,B00DVE8DUG,B00DVM34NO,B00EAHYZD2,B00FA2RLX2,B00GLL98Y6,B00GV0TGQW,B00HA2ZKCO,B00I0HQ4MS,B00I8XZ4HA,B00ILI7SVW,B00IOGK4RG,B00J2QLCFU,B00J39OM2Q,B00J5HDEPC,B00JQDFSGS
0,B00004S9I0,1.000000,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.032305,-0.022727,-0.022727,-0.032171,-0.022727,1.000000,-0.038668,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040291,-0.022727,0.564076,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.046772,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,...,-0.022727,-0.022727,-0.032305,-0.022727,-0.032513,-0.022727,-0.047088,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040073,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,-0.032171,-0.040291,-0.022727,-0.022727,-0.022727
1,B00004U9J9,-0.022727,1.000000,-0.022727,1.000000,-0.022727,-0.022727,-0.022727,-0.032305,-0.022727,-0.022727,-0.032171,-0.022727,-0.022727,-0.038668,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040291,-0.022727,-0.040291,-0.022727,-0.022727,1.000000,-0.022727,-0.022727,-0.022727,-0.022727,-0.046772,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,...,-0.022727,-0.022727,-0.032305,-0.022727,-0.032513,-0.022727,-0.047088,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040073,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,-0.032171,-0.040291,-0.022727,-0.022727,-0.022727
2,B0000534W2,-0.022727,-0.022727,1.000000,-0.022727,-0.022727,-0.022727,-0.022727,-0.032305,-0.022727,1.000000,-0.032171,-0.022727,-0.022727,-0.038668,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040291,-0.022727,-0.040291,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.046772,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,...,-0.022727,-0.022727,-0.032305,-0.022727,-0.032513,-0.022727,-0.047088,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,0.603953,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,0.588271,0.564076,-0.022727,-0.022727,-0.022727
3,B00005AXIY,-0.022727,1.000000,-0.022727,1.000000,-0.022727,-0.022727,-0.022727,-0.032305,-0.022727,-0.022727,-0.032171,-0.022727,-0.022727,-0.038668,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040291,-0.022727,-0.040291,-0.022727,-0.022727,1.000000,-0.022727,-0.022727,-0.022727,-0.022727,-0.046772,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,...,-0.022727,-0.022727,-0.032305,-0.022727,-0.032513,-0.022727,-0.047088,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.032513,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.022727,-0.040073,-0.032513,-0.022727,-0.0

In [44]:
df1.to_csv('./prod_ratings_sports.csv',index=False)

In [45]:
proddsdf = pd.read_csv('./prod_ratings_sports.csv')

In [46]:
proddsdf.head()

,userID,prod_ID,rating,prod_name,imgurl
0,A1M2T0J45TTE64,B00004S9I0,5.0,Lodge Original Finish Pro Grid/Iron Griddle,['https://images-na.ssl-images-amazon.com/imag...
1,AGGAQLXPVGM5D,B00004U9J9,5.0,SOG Pentagon Elite I Folding Knife PE14-CP - B...,['https://images-na.ssl-images-amazon.com/imag...
2,AK7CAW3N1XZV6,B0000534W2,5.0,"TheraBand Exercise Ball, Stability Ball with 6...",['https://images-na.ssl-images-amazon.com/imag...
3,AGGAQLXPVGM5D,B00005AXIY,5.0,Bushnell Yardage Pro Scout Laser Rangefinder,['https://images-na.ssl-images-amazon.com/imag...
4,A22CW0ZHY3NJH8,B00005BV0D,5.0,Liberty Mountain Aerobie Sprint,['https://images-na.ssl-images-amazon.com/imag...


In [47]:
proddsdf['imgurl'].loc[0][1]

"'"

In [48]:
proddsdf['imgurl'] = proddsdf['imgurl'].apply(lambda x: x.strip('][').split(', ') ) 

In [49]:
proddsdf['imgurl'].loc[0][0]

"'https://images-na.ssl-images-amazon.com/images/I/41KTD3MFQ2L._SS40_.jpg'"